In [44]:
from datetime import datetime
from collections import defaultdict
import calendar

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    # Parse the target month and calculate its start and end
    month_start = datetime.strptime(target_month + "-01", "%Y-%m-%d")
    days_in_month = calendar.monthrange(month_start.year, month_start.month)[1]
    month_end = month_start.replace(day=days_in_month)

    grouped = defaultdict(lambda: {"qty": 0.0, "amount": 0.0})
    total_revenue = 0.0

    for item in item_list:
        try:
            item_start = datetime.strptime(item["start_date"], "%Y-%m-%d")
            item_stop = datetime.strptime(item["stop_date"], "%Y-%m-%d")

            if item_stop < item_start:
                continue

            # Calculate active range for this item in the target month
            active_start = max(month_start, item_start)
            active_end = min(month_end, item_stop)

            if active_start > active_end:
                continue  # Not active in target month

            total_days = (item_stop - item_start).days + 1
            active_days = (active_end - active_start).days + 1

            qty = float(item["qty"])  
            amount = float(item["amount"])
            rate = float(item["rate"])

            daily_amount = amount / total_days
            prorated_amount = daily_amount * active_days

            billing_period = f"{active_start.date()} to {active_end.date()}"
            group_key = (item["item_code"], rate, billing_period)

            grouped[group_key]["qty"] += qty  
            grouped[group_key]["amount"] += prorated_amount

        except (ValueError, TypeError, ZeroDivisionError):
            continue

    line_items = []
    for (item_code, rate, billing_period), data in grouped.items():
        qty_rounded = round(data["qty"], 2)
        amount_rounded = round(data["amount"], 2)

        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": qty_rounded,
            "amount": amount_rounded,
            "billing_period": billing_period
        })

        total_revenue += amount_rounded

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [45]:
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {''
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": 10,
        "rate": 1080,
        "amount": 10800,
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": 1080,
        "amount": 16200,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": 1000,
        "amount": 5000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": 5000,
        "amount": 35000,
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": 8,
        "rate": 1100,
        "amount": 8800,
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": 3,
        "rate": 5200,
        "amount": 15600,
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": 20000,
        "amount": 20000,
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": 1000,
        "amount": 5000,
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": 7000,
        "amount": 14000,
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": 7000,
        "amount": 7000,
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
        "idx": 16,
        "item_code": "Breakout Area",
        "qty": 3,
        "rate": 3000,
        "amount": 9000,
        "start_date": "2024-01-01",
        "stop_date": "2024-01-31",
    }
]


In [59]:


bill = generate_monthly_bill(item_list, "2024-01")

from pprint import pprint
pprint(bill)


{'line_items': [{'amount': 880.68,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 10.0,
                 'rate': 1000.0},
                {'amount': 4051.14,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Manager Cabin',
                 'qty': 10.0,
                 'rate': 4600.0},
                {'amount': 9000.0,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Breakout Area',
                 'qty': 3.0,
                 'rate': 3000.0}],
 'total_revenue': 13931.82}
